In [13]:
import plotly.express as px

In [31]:
import requests
from bs4 import BeautifulSoup
header={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}

In [65]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

queries = [
    'forfait+gratuit+Mali+vpn',
    'vpn+mali',
    'anonytune_mali',
    'yourFreedom+orange',
    'internet_free+mali',
    'host_orangemali',
    'forfait+illimité+mali',
    'connexion+gratuit+mali',
    'tunnel+orange+Mali',
    'pasdeforfaitinternetorange',
    'apn+mali',
    'fraud+data+mali'
]

all_results = []

for query in queries:
    url = f'https://www.google.com/search?q={query}'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        results = []
        for i in soup.findAll('time'):
            if i.has_attr('datetime'):
                print(i['datetime'])
        for g in soup.find_all('div', {'class': 'tF2Cxc'}):
            anchors = g.find_all('a', limit=100000)
            if anchors:
                title = anchors[0].find('h3').text
                link = anchors[0]['href']
                soup2 = BeautifulSoup(title)
                dates = []
                for i in soup2.findAll(title):
                    if i.has_attr('datetime'):
                        print(i['datetime'])
                        dates.append(i['datetime'])
                # Extracting date information if available
                date_element = g.find('span', {'class': 'tddYDc'})
                date = dates

                description_elem = g.find('div', {'class': 'IsZvec'})
                description = description_elem.text.strip() if description_elem else "No description available"

                results.append([title, link, description, date])

        all_results.extend(results)

# Convert the results to a DataFrame
df = pd.DataFrame(all_results, columns=['Title', 'Link', 'Description', 'Date'])

# Save the DataFrame to a CSV file
df.to_csv('serp.csv', index=False, encoding='utf-8')

# Display the DataFrame
print(df)


                                                 Title   
0              Connexion internet gratuite Orange Mali  \
1    Internet gratuit illimité Orange Mali (Astuces...   
2                                  Vpn gratuit au Mali   
3       Comment avoir internet gratuit chez orangemali   
4              la connection internet gratuite au mali   
..                                                 ...   
96   Average Fraud Detection Supervisor Salary in B...   
97                                ASToN network's Post   
98   Move from Fraud Detection to Fraud Prevention ...   
99    Be on your guard: The rising risk of Data Frauds   
100        Financial Fraud Prevention, Graph Analytics   

                                                  Link   
0     https://www.facebook.com/groups/373261044156562/  \
1    https://www.gb00tech.com/2020/12/internet-grat...   
2    https://forums.commentcamarche.net/forum/affic...   
3    https://forums.commentcamarche.net/forum/affic...   
4            

In [61]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

queries = [
    'forfait+gratuit+Mali+vpn',
    'vpn+mali',
    'anonytune_mali',
    'yourFreedom+orange',
    'internet_free+mali',
    'host_orangemali',
    'forfait+illimité+mali',
    'connexion+gratuit+mali',
    'tunnel+orange+Mali',
    'pasdeforfaitinternetorange',
    'apn+mali',
    'fraud+data+mali'
]

all_results = []

for query in queries:
    url = f'https://www.google.com/search?q={query}'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        results = []
        for i in soup.findAll('time'):
            if i.has_attr('datetime'):
                print(i['datetime'])

In [67]:
date

[]

In [34]:
len(text)

101

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
text = []
for query in [
    'forfait+gratuit+Mali+vpn',
    'vpn+mali',
    'anonytune_mali',
    'yourFreedom+orange',
    'internet_free+mali',
    'host_orangemali',
    'forfait+illimité+mali',
    'connexion+gratuit+mali',
    'tunnel+orange+Mali',
    'pasdeforfaitinternetorange', 'apn+mali', 'fraud+data+mali'
] : 
    url = f'https://www.google.com/search?q={query}'
    header={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
    data = requests.get(url, headers=header)

    if data.status_code == 200:
        soup = BeautifulSoup(data.content, "html.parser")
        results = []
        for g in soup.find_all('div',  {'class':'g'}):
            anchors = g.find_all('a')
            if anchors:
                #link = anchors[0]['href']
                title = g.find('a').text
                try:
                    description = g.find('div', {'data-sncf':'2'}).text
                    text_desc = text_info.find('div', attrs={'class':'card-content t-small bt p20'}).getText(strip=True)
                except Exception as e:
                    description = "-"
                    text_desc = '-'
            results.append(str(title)+";"+str(link)+';'+str(description)+';'+str(text_desc))

    with open("serp.csv", "w") as f:
        f.write("Title; Link; Description\n")
    for result in results:
        with open("serp.csv", "a", encoding="utf-8") as f:
            f.write(str(result)+"\n")
        print(result)
        text.append(result)

In [16]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

queries = [
    'forfait+gratuit+Mali+vpn',
    'vpn+mali',
    'anonytune_mali',
    'yourFreedom+orange',
    'internet_free+mali',
    'host_orangemali',
    'forfait+illimité+mali',
    'connexion+gratuit+mali',
    'tunnel+orange+Mali',
     'pasdeforfaitinternetorange', 'apn+mali', 'fraud+data+mali'
]

all_results = []

for query in queries:
    url = f'https://www.google.com/search?q={query}'
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
    data = requests.get(url, headers=header)

    if data.status_code == 200:
        soup = BeautifulSoup(data.content, 'html.parser')
        results = []
        for g in soup.find_all('div', {'class': 'tF2Cxc'}):
            anchors = g.find_all('a')
            if anchors:
                title = anchors[0].find('h3').text
                link = anchors[0]['href']
                description_elem = g.find('div', {'class': 'IsZvec'})
                description = description_elem.text if description_elem else "No description available"
                results.append([title, link, description])

        all_results.extend(results)

# Convert the results to a DataFrame
df = pd.DataFrame(all_results, columns=['Title', 'Link', 'Description'])

# Save the DataFrame to a CSV file
df.Title.to_csv('serp.csv', index=False, encoding='utf-8')

# Display the DataFrame
print(df)


                                                Title   
0             Connexion internet gratuite Orange Mali  \
1   Internet gratuit illimité Orange Mali (Astuces...   
2                                 Vpn gratuit au Mali   
3        Comment configurer internet sur votre mobile   
4             la connection internet gratuite au mali   
..                                                ...   
95  Determinants of Customs Fraud and Corruption: ...   
96  Average Fraud Detection Supervisor Salary in B...   
97  Move from Fraud Detection to Fraud Prevention ...   
98                               ASToN network's Post   
99   Be on your guard: The rising risk of Data Frauds   

                                                 Link   
0    https://www.facebook.com/groups/373261044156562/  \
1   https://www.gb00tech.com/2020/12/internet-grat...   
2   https://forums.commentcamarche.net/forum/affic...   
3   https://www.orangemali.com/fr/comment-configur...   
4                        http:

In [18]:
from itertools import islice
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import pandas as pd
import matplotlib.pyplot as plt

def load_text(filename):
    my_text = list()
    with open(filename, encoding= "utf-8") as f:
        for line in islice(f, 0, None):
            my_text.append(line)
    my_text = [word_tokenize(sentence) for sentence in my_text]
    flat_list = [item for sublist in my_text for item in sublist]
    return flat_list

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diagne080894\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\diagne080894\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
goog = load_text('serp.csv')
goog

['Title',
 'Connexion',
 'internet',
 'gratuite',
 'Orange',
 'Mali',
 'Internet',
 'gratuit',
 'illimité',
 'Orange',
 'Mali',
 '(',
 'Astuces',
 '2021',
 ')',
 'Vpn',
 'gratuit',
 'au',
 'Mali',
 'Comment',
 'configurer',
 'internet',
 'sur',
 'votre',
 'mobile',
 'la',
 'connection',
 'internet',
 'gratuite',
 'au',
 'mali',
 'NOUVEAU',
 'CONNECTION',
 'GRATUIT',
 'PAR',
 'ORANGEMALI',
 'Orange',
 'Mali',
 'Sugu',
 '–',
 'Applications',
 'sur',
 '...',
 'Orange',
 'Mali',
 '(',
 '@',
 'Orange_Mali',
 ')',
 '/',
 'X',
 '``',
 "J'ai",
 'besoin',
 "d'un",
 'vpn',
 'gratuit',
 'chez',
 'orange',
 'mali',
 ',',
 'qui',
 'block',
 '...',
 "''",
 'VPN',
 'pour',
 'le',
 'Mali',
 '-',
 'Quel',
 'est',
 'le',
 'Meilleur',
 '?',
 'Meilleur',
 'VPN',
 'pour',
 'le',
 'Mali',
 '-',
 'VPNMag',
 'Mali',
 'VPN',
 '-',
 'Fast',
 'Private',
 'Proxy',
 '-',
 'Apps',
 'on',
 'Google',
 'Play',
 'Mali',
 'VPN',
 '-',
 'Get',
 'Fast',
 '&',
 'Free',
 'Mali',
 'IP',
 'APK',
 '-',
 'Dernière',
 'version',

In [20]:
def prepare_text(list_of_words):
  #load stopwords:
  stops = stopwords.words('english')
  #transform all word characters to lower case:
  list_of_words = [word.lower() for word in list_of_words]
  #remove all words containing up to two characters:
  list_of_words = [word for word in list_of_words if len(word)>2]
  #remove stopwords:
  list_of_words = [word for word in list_of_words if word not in stops]
  return list_of_words

In [21]:
goog_process = prepare_text(goog)
goog_process

['title',
 'connexion',
 'internet',
 'gratuite',
 'orange',
 'mali',
 'internet',
 'gratuit',
 'illimité',
 'orange',
 'mali',
 'astuces',
 '2021',
 'vpn',
 'gratuit',
 'mali',
 'comment',
 'configurer',
 'internet',
 'sur',
 'votre',
 'mobile',
 'connection',
 'internet',
 'gratuite',
 'mali',
 'nouveau',
 'connection',
 'gratuit',
 'par',
 'orangemali',
 'orange',
 'mali',
 'sugu',
 'applications',
 'sur',
 '...',
 'orange',
 'mali',
 'orange_mali',
 "j'ai",
 'besoin',
 "d'un",
 'vpn',
 'gratuit',
 'chez',
 'orange',
 'mali',
 'qui',
 'block',
 '...',
 'vpn',
 'pour',
 'mali',
 'quel',
 'est',
 'meilleur',
 'meilleur',
 'vpn',
 'pour',
 'mali',
 'vpnmag',
 'mali',
 'vpn',
 'fast',
 'private',
 'proxy',
 'apps',
 'google',
 'play',
 'mali',
 'vpn',
 'get',
 'fast',
 'free',
 'mali',
 'apk',
 'dernière',
 'version',
 'get',
 'best',
 'mali',
 'vpn',
 'swift',
 'secure',
 'recrutement',
 'vpn',
 'mali',
 'mali',
 'super',
 'vpn',
 'free',
 'fast',
 'secure',
 'unlimited',
 '...',
 'vpn

In [22]:
########Function to count of words
unique_words = []
counts = []
def count_freq(my_list) : 
    for word in my_list : 
        if word not in unique_words : 
            unique_words.append(word) 
    for i in unique_words : 
        count = 0
        for j in my_list :
            if i == j :
                count+=1
        counts.append(count)
    df = pd.DataFrame({"word": unique_words, "count": counts})
    df.sort_values(by="count", inplace = True, ascending = False)
    df.reset_index(drop=True, inplace = True)
    return df

In [23]:
goog_process = count_freq(goog_process)
goog_process = goog_process.iloc[:10]

In [24]:
goog_process.to_csv('goog_process.csv')

PermissionError: [Errno 13] Permission denied: 'goog_process.csv'

In [ ]:
fig = px.bar(goog_process, x = 'word', y = 'count',
                width=1400, height=800)
fig.update_layout(
xaxis_title='Name groups',
yaxis_title='Number of members',
plot_bgcolor='white',  # Transparent plot background
paper_bgcolor='rgb(255,255,255)',  # White background
font=dict(family='Arial', size=12, color='blue'),  # Font style
margin=dict(l=50, r=50, t=50, b=50),  # Setting margins
xaxis=dict(tickangle=45),  # Rotating x-axis labels
yaxis=dict(tickformat=',d'),  # Adding comma to y-axis labels for thousands separator, 
title = '📊 Graph III : Nb_members : Top 10 group',
title_x = 0.4
)

In [25]:
fig = px.bar(goog_process, x = 'word', y = 'count', 
width=1400, height=800 )
fig.update_layout(
xaxis_title='words',
yaxis_title='Frequency of word',
plot_bgcolor='white',  # Transparent plot background
paper_bgcolor='rgb(100,200,200)',  # White background
font=dict(family='Arial', size=12, color='black'),  # Font style
margin=dict(l=50, r=50, t=50, b=50),  # Setting margins
xaxis=dict(tickangle=45),  # Rotating x-axis labels
yaxis=dict(tickformat=',d'),  # Adding comma to y-axis labels for thousands separator, 
title = '📊 TOP 15 of words',
title_x = 0.4
)

In [43]:
>>> from bs4 import BeautifulSoup
>>> s = '''<time class="jlist_date_image" datetime="2015-04-02 14:30:12">Idag <span class="list_date">14:30</span></time>'''
>>> soup = BeautifulSoup(s)



2015-04-02 14:30:12


In [42]:
df

,Title,Link,Description,Date,Year
0,Connexion internet gratuite Orange Mali,https://www.facebook.com/groups/373261044156562/,No description available,No date available,No year available
1,Internet gratuit illimité Orange Mali (Astuces...,https://www.gb00tech.com/2020/12/internet-grat...,No description available,No date available,No year available
2,Vpn gratuit au Mali,https://forums.commentcamarche.net/forum/affic...,No description available,No date available,No year available
3,la connection internet gratuite au mali,http://laban.centerblog.net/,No description available,No date available,No year available
4,Comment avoir internet gratuit chez orangemali,https://forums.commentcamarche.net/forum/affic...,No description available,No date available,No year available
...,...,...,...,...,...
95,Average Fraud Detection Supervisor Salary in B...,https://worldsalaries.com/average-fraud-detect...,No description available,No date available,No year available
96,ASToN network's Post,https://ml.linkedin.com/posts/aston-network_ba...,No description available,No date available,No year available
97,Move from Fraud Detection to Fraud Prevention ...,http://www.teradata.fr/Solutions/Fraud-Prevention,No description available,No date available,No year available
98,Be on your guard: The rising risk of Data Frauds,https://www.subex.com/blog/be-on-your-guard-th...,No description available,No date available,No year available
